In [ ]:
from simulations.drosselschwab import simulate_drosselschwab
import results
import data

# Then when saving a file you can use the path function like so:
results_path = results.path('p_param_2.csv')
data_path = data.path('p_param_2.csv')
print(results_path, data_path)

In [3]:
L = 256
p = 0.01
f = 0.0001
steps = 10000

print("Start experiment:")

fire_sizes, grid = simulate_drosselschwab(L=L, p=p, f=f, steps=steps)

Start experiment:
